# Planning

- Selitettävyys on asiakkaalle tärkeätä, joten mennään simppeleillä malleilla. (Tavoite ei ole ennustaa vaan selittää)
- Asiakkaan mukaan vaste ei ole lineaarinen, joten päätöspuut voivat olla parempia kuin lin. regressio.
- Kokeile seuraavia malleja sk-learnista:
  1. Peruspäätöspuu
  1. Random forest
  1. Lineaarinen regressio (vertailukohta, kertoimet helppo tulkita)
- Kohdemuuttuja (y):
  - Joko neljän kohdemuuttujan yhteenlaskettu päiväkohtainen z-score tai
  - Kunkin muuttujan z-score erikseen (neljä eri mallia)
  - Testaile tehtyjen valintojen (mean vs. median, luottamusvälin leveys, ikkunoiden pituus, 10-14 vs. koko päivä), vaikutusta
  - Syötä data ei-aikasarjamuotoisena. Eli niin, että kukin arvo on itsenäinen. Näin dataa voi sekoittaa eikä tarvitse edetä aikajärjestyksessä
    - Tavoite ei ole ennustaa seuraavan askeleen y:tä edellisestä askeleesta.
  
- Selittävät muuttujat (x):
  - Light
  - Rain
  - Temperature
  - Soil water potential
  - Soil water content
  - Päivä (=vuoden aika) -> muuta päivänumero (1-365) radiaaneiksi, jotta joulukuu ja tammikuu tulevat vierekkäin? 
  - Kokeile näiden lisäksi erilaisia time lageja, sillä intuitiivisesti tuntuisi, että vaste ei välttämättä ole aivan välitön. Esim. kolmen edellisen päivän keskiarvo tms.
    - Älä siis lagaa y:tä, mutta x:iä voi lagata

# Imports

In [4]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
from sklearn import tree # https://scikit-learn.org/stable/modules/tree.html#regression
from sklearn.ensemble import RandomForestRegressor # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html